<a href="https://colab.research.google.com/github/addis0nl/YTSummarise/blob/main/YTSummarise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Summarise YouTube videos w/ LLM via built-in transcriptions

This colab notebook uses 13b model instead of 7b.

Make sure to select a GPU runtime.

## Setup

In [ ]:
!pip install youtube_transcript_api
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python
!wget https://huggingface.co/TheBloke/LlongOrca-13B-16K-GGML/resolve/main/llongorca-13b-16k.ggmlv3.q4_K_S.bin

from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from llama_cpp import Llama

# Change function to suit model if neccessary
def get_path():
  return "llongorca-13b-16k.ggmlv3.q4_K_S.bin"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 5.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.78-cp310-cp310-linux_x86_64.whl size=5822262 sha256=98e63520e88252548fa5a32773c53a75d7bc1c24aabf1a616b0846019c788203
  Stored in directory: /root/.cache/pip/wheels/61/f9/20/9ca660a9d3f2a47e44217059409478865948b5c8a1cba70030
Successfully built llama-cpp-python
--2023-08-25 09:04:57--  https://huggingface.co/TheBloke/LlongOrca-13B-16K-GGML/resolve/main/llongorca-13b-16k.ggmlv3.q4_K_S.bin
Resolving huggingface.co (huggingface.co)... 65.9.86.57, 65.9.86.79, 65.9.86.62, ...
Connecting to huggingface.co (huggingface.co)|65.9.86.57|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.

In [ ]:
def sysprompt():
    return "### System: You are an AI assistant that summarises videos based on it's audio transcript. You only use information and context from the transcript, in other words, you do not state anything that is not explicitly stated in the transcript.\n\n"

def quick_llm(text):
    llama = Llama(model_path=get_path(), n_gpu_layers=-1, seed=-1, n_ctx=4096, rope_freq_scale=0.5)
    if detailed:
        for sections in text:
            section = ''.join(sections)
            output = llama(sysprompt()+'### Instruction:\n\nList the details from the video shown here:\n"' +section +
                           '"\n\n### Response:\n', max_tokens=2048, temperature=0.5, stream=stream)
            display(output)
    else:
        output = llama(sysprompt()+'### Instruction:\n\nSummarise the following concisely:\n"' + text + '"\n\n### Response:\n',
                     max_tokens=2048, stop=['###'], stream=stream, temperature=0.4)
        display(output)

def llm(text):
    llama = Llama(model_path=get_path(), n_gpu_layers=40, seed=-1, n_ctx=16384, rope_freq_scale=0.25)
    output = llama(sysprompt()+'### Instruction:\n\nAccurately summarise all important information from the following:\n"'
                   + text + '"\n\n### Response:\n', max_tokens=4096, stop=['###'], stream=stream, temperature=0.3)
    display(output)

def out(text):
    outfile = "output/" + video_id + ".txt"
    with open(outfile, "a") as f:
        f.write(text+"\n")

def display(output):
    if stream:
        for x in output:
            print(x['choices'][0]['text'], end="")
    else:
        print(output['choices'][0]['text'], end="")
        if out:
            out(output['choices'][0]['text'])

def get_transcript(id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(id)
    except TranscriptsDisabled:
        return None
    length = transcript[-1]['start'] + transcript[-1]['duration']
    n_segments = int(length / 60 / 5)
    segments = [[] for i in range(n_segments + 1)]
    for line in transcript:
        segments[int(line['start'] / 60 / 5)].append(line['text'] + " ")
    return segments

def combine_sub(segments):
    return ''.join([''.join(s) for s in segments])

def summary(segments):
    all_text = combine_sub(segments)
    if long_video:
        mid = len(segments)//2
        a = combine_sub(segments[:mid])
        b = combine_sub(segments[mid:])
        print("\n\nPart 1 of 2:\n\n")
        llm(a)
        print("\n\nPart 2 of 2:\n\n")
        llm(b)
    elif detailed:
        quick_llm(segments)
    else:
        all_text = combine_sub(segments)
        if quick:
            quick_llm(all_text)
        else:
            llm(all_text)

# Inference

**To Use:**

1. In the ID field below, enter the alphanumeric id of your chosen YouTube video, i.e. youtu.be/[id] or youtube.com/watch?v=[id]
2. Choose optional parameters
3. Run cell and again if necessary

**Optional Parameters:**

* *stream* - Output text as soon as it's created
* *quick* - Use smaller context LLM for faster inference
* *detailed* - Summarise every 5 minute chunk of the video
* *long_video* - Use larger context LLM for longer videos (experimental)
* *out* - Also output into .txt file, can be used with transcript option
* *transcript* - Output full transcript only, no summarisation

In [ ]:
ID = "dQw4w9WgXcQ" # @param {type:"string"}

stream = False # @param {type:"boolean"}
quick = False # @param {type:"boolean"}
detailed = False # @param {type:"boolean"}
long_video = False # @param {type:"boolean"}
out = False # @param {type:"boolean"}
transcript = False # @param {type:"boolean"}

if __name__ == '__main__':
    video_id = ID
    subs = get_transcript(video_id)
    if transcript:
        transcript = combine_sub(subs)
        print(transcript)
        if out:
            out(transcript)
    else:
        summary(subs)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


The video features a song with the lyrics, "Thank you we're no strangers to love." The singer expresses their feelings and wants to make the listener understand. They mention not being like any other guy and promise never to let the listener down.